In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors
from collections import Counter

import warnings
warnings.filterwarnings('ignore')


In [2]:
class Recommender:

    def __init__(self,profiles,recent_activity,dataset):
        self.df = dataset
        self.profiles = profiles
        self.recent_activity = recent_activity

    def get_features(self,dataframe):
        #getting dummies of dataset
        nutrient_dummies = dataframe.Nutrient.str.get_dummies()
        disease_dummies = dataframe.Disease.str.get_dummies(sep=' ')
        diet_dummies = dataframe.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)

        return feature_df

    def find_neighbors(self,dataframe,features,k):
        features_df = self.get_features(dataframe)
        total_features = features_df.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in features:
            d[i] = 1
        final_input = list(d.values())

        similar_neighbors = self.k_neighbor([final_input],features_df,dataframe,k)
        return similar_neighbors

    def k_neighbor(self,inputs,feature_df,dataframe,k):

        #initializing model with k neighbors
        model = NearestNeighbors(n_neighbors=k,algorithm='ball_tree')


        # fitting model with dataset features
        model.fit(feature_df)


        df_results = pd.DataFrame(columns=list(dataframe.columns))

        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(dataframe.loc[i])

        df_results = df_results.reset_index(drop=True)
        return df_results

    def user_based(self,features,user_id):

        similar_users = self.find_neighbors(self.profiles,features,10)
        users = list(similar_users.User_Id)

        results = self.recent_activity[self.recent_activity.User_Id.isin(users)] #taking acitivies

        results = results[results['User_Id']!=user_id] # selecting those which are not reviewed by user

        meals = list(results.Meal_Id.unique())

        results = self.df[self.df.Meal_Id.isin(meals)]

        results = results.filter(['Meal_Id','Name','Nutrient','Veg_Non','description','Price','Review'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results

    def recent_activity_based(self,user_id):
        recent_df = self.recent_activity[self.recent_activity['User_Id']==user_id]
        meal_ids = list(recent_df.Meal_Id.unique())
        recent_data = self.df[self.df.Meal_Id.isin(meal_ids)][['Nutrient','catagory','Disease','Diet']].reset_index(drop=True)

        disease = []
        diet = []
        for i in range(recent_data.shape[0]):
            for j in recent_data.loc[i,'Disease'].split():
                disease.append(j)
        for i in range(recent_data.shape[0]):
            for j in recent_data.loc[i,'Diet'].split():
                diet.append(j)

        value_counts = recent_data.Nutrient.value_counts()
        m = recent_data.Nutrient.value_counts().mean()
        features = list(value_counts[recent_data.Nutrient.value_counts()>m].index)
        a = dict(Counter(disease))

        m = np.mean(list(a.values()))
        for i in a.items():
            if i[1]>m:
                features.append(i[0])
        a = dict(Counter(diet))
        m = np.mean(list(a.values()))
        for i in a.items():
            if i[1]>m:
                features.append(i[0])

        similar_neighbors = self.find_neighbors(self.df,features,10)
        return similar_neighbors.filter(['Meal_Id','Name','Nutrient','Veg_Non','description','Price','Review'])

    def recommend(self,user_id):
        #finding user's profile features by id
        profile = self.profiles[self.profiles['User_Id']==user_id]
        features = []
        features.append(profile['Nutrient'].values[0])
        features.extend(profile['Disease'].values[0].split())
        features.extend(profile['Diet'].values[0].split())
        df1 = self.user_based(features,user_id)

        df2 = self.recent_activity_based(user_id)
        df = pd.concat([df1,df2])

        df = df.drop_duplicates('description').reset_index(drop=True)
        return df

In [3]:
from google.colab import files
uploaded = files.upload()

Saving user_Profiles.csv to user_Profiles.csv
Saving recent_activity.csv to recent_activity.csv
Saving dataset.csv to dataset.csv


-------ROUGH -----
'sample user','non-veg','calcium','eye_disease','high_protein_diet high_fat_diet'-------

In [17]:
from csv import writer
Input_val = []
List = ['name : ','veg/non-veg : ','nutrient type : ','afflicted disease if any : ','diet type : ']
for i in range(5):
  Input_val.append(input(List[i]))

with open('user_Profiles.csv','a') as f_object:
  w_obj = writer(f_object)
  w_obj.writerow(Input_val)
  f_object.close()


name : Sanjay
veg/non-veg : veg
nutrient type : calcium
afflicted disease if any : animea
diet type : protein


In [19]:
user_id = 'User_name'  # name of current user

profiles = pd.read_csv('user_Profiles.csv') # profiles of all users
recent_activity = pd.read_csv('recent_activity.csv') # recent activities of current user (meals liked,rated,searched,Purchased)
dataset = pd.read_csv('dataset.csv') # main dataset


ob = Recommender(profiles,recent_activity,dataset)
result = ob.recommend(user_id)
result

,Meal_Id,Name,Nutrient,Veg_Non,description,Price
0,meal_id30,couscous with ratatouille - tangy tomato sauce,chloride,veg,"for the cous cous:, plain couscous, extra virg...",220
1,meal_id50,belgian pork chop,iron,veg,"pork chop, pink pepper corn, green pepper corn...",215
2,meal_id36,spicy watermelon soup,sodium,veg,"तरबूज, अदरक-लहसुन का पेस्ट, पुदीना, चिली फलेक्...",225
3,meal_id223,veg summer rolls,carbohydrates,veg,"rice paper sheets, iceberg lettuce, carrot, be...",545
4,meal_id127,cajun spiced turkey wrapped with bacon,vitamin_c,non-veg,"turkey breast, cajun spice, spinach leaves (co...",635
5,meal_id275,mexican pizza,sodium,veg,"आटा टॉर्टिया, रिफ्राइड बीन्स, बेल पेपर, स्प्रि...",255
6,meal_id187,tricolour pizza,sodium,veg,"pizza base , pizza sauce, mozzarella cheese, b...",380
7,meal_id68,gajar halwa tart,vitamin_d,veg,"white butter, breakfast sugar, milk full fat, ...",205
8,meal_id86,roast turkey with cranberry sauce,chloride,non-veg,"whole turkey, butter, onion, celery, crumbled ...",630
9,meal_id239,homemade gulab jamun,vitamin_d,veg,"sugar, water, milk, cardamom seeds, saffron, c...",445


User_100
